# Plotting

This section introduces UXarray's plotting API, showcasing how to visualize both data variables and grid topology.


In [1]:
import uxarray as ux
import holoviews as hv

## Data

For most of the examples in this notebook, we will be using a simple mesh consisting of 4 hexagons, with sample data mapped to the faces, edges, and nodes.

In [2]:
grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"

data_paths = [
    "../../test/meshfiles/ugrid/quad-hexagon/random-node-data.nc",
    "../../test/meshfiles/ugrid/quad-hexagon/random-edge-data.nc",
    "../../test/meshfiles/ugrid/quad-hexagon/random-face-data.nc",
]

uxds = ux.open_mfdataset(grid_path, data_paths)

uxds

<xarray.UxDataset> Size: 312B
Dimensions:           (n_edge: 19, n_face: 4, n_node: 16)
Dimensions without coordinates: n_edge, n_face, n_node
Data variables:
    random_data_edge  (n_edge) float64 152B dask.array<chunksize=(19,), meta=np.ndarray>
    random_data_face  (n_face) float64 32B dask.array<chunksize=(4,), meta=np.ndarray>
    random_data_node  (n_node) float64 128B dask.array<chunksize=(16,), meta=np.ndarray>

TODO:

## Grid Topology Visualization 

In [3]:
uxgrid = uxds.uxgrid
uxgrid

<uxarray.Grid>
Original Grid Type: UGRID
Grid Dimensions:
  * n_node: 16
  * n_edge: 19
  * n_face: 4
  * n_max_face_nodes: 6
  * two: 2
  * n_nodes_per_face: (4,)
Grid Coordinates (Spherical):
  * node_lon: (16,)
  * node_lat: (16,)
  * edge_lon: (19,)
  * edge_lat: (19,)
  * face_lon: (4,)
  * face_lat: (4,)
Grid Coordinates (Cartesian):
Grid Connectivity Variables:
  * face_node_connectivity: (4, 6)
  * edge_node_connectivity: (19, 2)
Grid Descriptor Variables:
  * n_nodes_per_face: (4,)

In [4]:
uxgrid.plot(title="Default Grid Plot")

TypeError: sequence item 0: expected str instance, NoneType found

### Edges

In [5]:
uxgrid.plot.edges(color="black", title="Grid Edge Plot")

ValueError: Unsupported backend. Expected one of ['bokeh', 'matplotlib'], but received None

### Points

In [ ]:
(
    uxgrid.plot.edges(color="black")
    * uxgrid.plot.nodes(marker="o", size=100).relabel("Corner Nodes")
    * uxgrid.plot.face_centers(marker="s", size=100).relabel("Face Centers")
    * uxgrid.plot.edge_centers(marker="^", size=100).relabel("Edge Centers")
).opts(title="Grid Coordinates")

## Data Visualization


## Visualizing  Data

If you are working with data that resides on an unstructured grid, visualization is done through a `UxDataArray`


### Default `UxDataArray` Plot

In [ ]:
uxds["random_data_face"].plot(cmap=ux.cmaps.diverging, title="Default UxDataArray Plot")

### Polygons

In [ ]:
(
    uxds["random_data_face"].plot.polygons(
        rasterize=False,
        cmap=ux.cmaps.diverging,
        title="Vector Polygons",
        colorbar=False,
    )
    + uxds["random_data_face"].plot.polygons(
        rasterize=True, cmap=ux.cmaps.diverging, title="Raster Polygons", colorbar=False
    )
).cols(1)

### Points

TODO

In [ ]:
hv.extension("bokeh")
(
    uxds.uxgrid.plot(line_color="black")
    * uxds["random_data_node"]
    .plot.points(
        cmap="inferno", size=100, marker="circle", clabel=None, tools=["hover"]
    )
    .relabel("Node Data")
    * uxds["random_data_edge"]
    .plot.points(
        cmap="inferno", size=100, marker="square", clabel=None, tools=["hover"]
    )
    .relabel("Edge Data")
    * uxds["random_data_face"]
    .plot.points(
        cmap="inferno", size=100, marker="triangle", clabel=None, tools=["hover"]
    )
    .relabel("Face Data")
).opts(title="Plotting Data as Points")

## Working with Periodic Data

The dataset used in the previous examples is an extremely simple grid, consisting only of 4 hexagons. When working with a global grid that is periodic, faces that wrap around the antimeridian will cause visual artifacts unless they are corrected. The ``periodic_elements`` parameter can be used to determine how to handle these elements.
* ``periodic_elements='exclude'``: Periodic polygons are excluded from the final plot
* ``periodic_elements='split'``: Periodic polygons are split along the antimeridian
* ``periodic_elements='ignore'``: Periodic polygons are ignored

```{warning}
It is suggested to keep ``periodic_elements='exclude'`` when working with moderatly large datasets, as there is a significant overhead needed correct the antimeridian faces.
```


In [ ]:
base_path = "../../test/meshfiles/mpas/QU/"
grid_path = base_path + "oQU480.231010.nc"
uxds_mpas = ux.open_dataset(grid_path, grid_path)

In [ ]:
(
    uxds_mpas["bottomDepth"]
    .plot(cmap=ux.cmaps.sequential_blue)
    .opts(width=700, height=350, title="Default Plot (Excluding Periodic Elements)")
    + uxds_mpas["bottomDepth"]
    .plot(
        periodic_elements="split", cmap=ux.cmaps.sequential_blue, width=700, height=350
    )
    .opts(title="Include Periodic Elements (Split)")
).cols(1)

## Geographic Projections & Features

UXarray's plotting API supports Cartopy Projections and Geographic features through the use of the ``GeoViews`` library. 


In [ ]:
import cartopy.crs as ccrs
import geoviews.feature as gf

In [ ]:
uxds_mpas["bottomDepth"].plot(
    projection=ccrs.Orthographic(),
    cmap=ux.cmaps.sequential_blue,
    title="Projected Polygon Plot",
) * gf.coastline(projection=ccrs.Orthographic())

### Shifting Center Coordinates

If you are using a projection with a shifted central longitude, you must use the ``Grid.set_central_longitude()`` function instead of the ``central_longitude`` parameter in the Projection.

```{warning}
The ``Grid.set_central_longitude()`` method performs an in-place operation. If you want to revert back to the original extent, you can run ``Grid.set_central_longitude(0.0)``
```

In [14]:
central_longitude = 180

uxds_mpas.uxgrid.set_central_longitude(central_longitude)

uxds_mpas["bottomDepth"].plot.polygons(
    rasterize=True,
    projection=ccrs.Orthographic(),
    cmap=ux.cmaps.sequential_blue,
    title="Projected Polygon Plot (Centered about 180 degrees longitude)",
) * gf.coastline(projection=ccrs.Orthographic(central_longitude=central_longitude))

:DynamicMap   []
   :Overlay
      .Image.I     :Image   [Longitude,Latitude]   (Longitude_Latitude bottomDepth)
      .Coastline.I :Feature   [Longitude,Latitude]